In [1]:
#   @@@@@@@     @      @    @  @   #
#   @  @  @    @ @      @  @       #
#      @  @   @   @      @@    @   #
#      @     @@@@@@@    @  @   @   #
#      @    @       @  @    @  @   #

Привет, в этой практике мы с вами применим наши знания по PySpark и постараемся изучить что-то новое в процессе выполнения.
<br>В занятии используется датасет собранный на основе данных <a href="https://www.kaggle.com/chicago/chicago-taxi-rides-2016">Chicago Taxi Rides 2016</a>
<br>Полная <a href="https://spark.apache.org/docs/latest/api/python/index.html">документация PySpark</a>.
<br>Схема данны:
<br>|-- taxi_id = идентификатор таксиста
<br>|-- trip_start_timestamp = время начала поездки
<br>|-- trip_end_timestamp = время окончания поездки
<br>|-- trip_seconds = время длительности поездки в секундах
<br>|-- trip_miles = мили проиденные во время поездки
<br>|-- fare = транспортные расходы
<br>|-- tips = назначенные чаевые
<br>|-- trip_total = общая стоимость поездки
<br>|-- payment_type = тип оплаты

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import round, col

In [3]:
spark = SparkSession.builder.appName('PySparkTasks').getOrCreate()

In [4]:
spark.conf.set("spark.sql.session.timeZone", "GMT+3")

In [5]:
spark

Скачайте <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week3/spark-tasks/taxi_data.parquet">taxi_data.parquet</a> и загрузите используя <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame">SparkAPI</a>

In [6]:
df =  spark.read.parquet('taxi_data.parquet') #Ваш код загрузки

№1 Посчитайте количество загруженных строк.

In [7]:
df.count() # Число строк

2540712

In [11]:
df.show()

+-------+--------------------+-------------------+------------+----------+-----+-----+----------+------------+
|taxi_id|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles| fare| tips|trip_total|payment_type|
+-------+--------------------+-------------------+------------+----------+-----+-----+----------+------------+
|   5240| 2016-12-15 23:45:00|2016-12-16 00:00:00|         900|       2.5|10.75| 2.45|      14.7| Credit Card|
|   1215| 2016-12-12 07:15:00|2016-12-12 07:15:00|         240|       0.4|  5.0|  3.0|       9.5| Credit Card|
|   3673| 2016-12-16 16:30:00|2016-12-16 17:00:00|        2400|      10.7| 31.0|  0.0|      31.0|        Cash|
|   5400| 2016-12-16 08:45:00|2016-12-16 09:00:00|         300|       0.0| 5.25|  2.0|      7.25| Credit Card|
|   1257| 2016-12-03 18:45:00|2016-12-03 18:45:00|         360|       0.3|  5.0|  0.0|       5.0|        Cash|
|   4666| 2016-12-30 18:00:00|2016-12-30 18:45:00|        2400|      18.2|46.75|10.15|      61.4| Credit Card|
|

Посмотрим схему данных:

In [13]:
df.printSchema()

root
 |-- taxi_id: integer (nullable = true)
 |-- trip_start_timestamp: timestamp (nullable = true)
 |-- trip_end_timestamp: timestamp (nullable = true)
 |-- trip_seconds: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- fare: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- trip_total: double (nullable = true)
 |-- payment_type: string (nullable = true)



№2 Чему равна корреляция и ковариация между длиной маршрута и ценой за поездку? Ответ округлите до 5 знаков после запятой.
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.corr">corr</a> & <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.cov">cov</a>

In [19]:
round(df.corr('trip_miles','trip_total'), 5) # Ваш код

0.44816

In [20]:
round(df.cov('trip_miles','trip_total'), 5)

71.96914

№3 Найдите количество, среднее, cреднеквадратическое отклонение, минимум и максимум для длины маршрута и цены за поездку? Ответ округлите до 1 знака после запятой. Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.describe">describe</a>

In [31]:
df.describe(['trip_miles','trip_total']).show()# Ваш код

+-------+------------------+------------------+
|summary|        trip_miles|        trip_total|
+-------+------------------+------------------+
|  count|           2540677|           2540672|
|   mean|3.0005873828090266|15.913560215564042|
| stddev|  5.25716922943536|30.546699217618237|
|    min|               0.0|               0.0|
|    max|             900.0|           9276.69|
+-------+------------------+------------------+



In [32]:
# todo огруглить https://stackoverflow.com/questions/47046827/trouble-with-pyspark-round-function
# type(df.describe(['trip_miles','trip_total']))

№4 Найдите самый НЕ популярный вид оплаты.
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy">groupBy</a> <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy">orderBy</a>

In [83]:
df.groupBy(['payment_type']).count().orderBy('count').collect()[0]# Ваш код

Row(payment_type='Way2ride', count=3)

№5 Найдите идентификатор таксиста выполнившего наибольшее число заказов.

In [47]:
df.groupBy('taxi_id').count().orderBy('count', ascending=False).collect()[0]

Row(taxi_id=316, count=2225)

№6 Чему равна средняя цена среди поездок, оплаченных наличными? Ответ округлите до 5 знака.
<br> Подробней <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.where">where</a>

In [105]:
ndf = df.filter("payment_type == 'Cash'")\
    .agg({"trip_total": "avg"})\
    .withColumnRenamed('avg(trip_total)', 'avg')

In [82]:
ndf.select("*", round(col('avg'),5)).show()

+------------------+-------------+
|               avg|round(avg, 5)|
+------------------+-------------+
|12.035261470840307|     12.03526|
+------------------+-------------+



In [59]:
#df.filter(df['payment_type'] == 'Cash').agg(avg(col("trip_total")))

№7 Сколько таксистов проехало больше 1000 миль за все время выполнения заказов?

In [106]:
df.groupBy(col('taxi_id'))\
    .agg({'trip_miles':'sum'})\
    .withColumnRenamed('sum(trip_miles)', 'sum')\
    .filter(col('sum') > 1000)\
    .count()

2860

№8 Сколько миль проехал пассажир в самой долгой поездке? (Ответ округлите до целого)

In [111]:
df.orderBy('trip_seconds', ascending=False)[['trip_miles']].show(3) # Ваш код

+----------+
|trip_miles|
+----------+
|       0.0|
|       0.0|
|       0.0|
+----------+
only showing top 3 rows



№9 Каков средний заработок всех таксистов? Ответ округлите до 5-ого знака.
<br>Отсеките неизвестные машины (не определенный taxi_id).

In [118]:
df.filter(df['taxi_id'].isNull() != True)\
    .groupBy('taxi_id')\
    .agg({'trip_total' : 'sum'})\
    .withColumnRenamed('sum(trip_total)', 'sum')\
    .agg({'sum':'avg'})\
    .show()
#.collect()[3]

+-----------------+
|         avg(sum)|
+-----------------+
|8218.856265256312|
+-----------------+



№10 Сколько поездок начиналось в самый загруженный час?
<br>Используйте функцию <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.hour">hour</a>

In [9]:
from pyspark.sql.functions import hour

In [14]:
df.groupBy(hour('trip_start_timestamp')).count().orderBy('count', ascending=False).show(5)

+--------------------------+------+
|hour(trip_start_timestamp)| count|
+--------------------------+------+
|                        18|181127|
|                        19|173779|
|                        17|169886|
|                        16|156519|
|                        20|146602|
+--------------------------+------+
only showing top 5 rows



№11 Сколько поездок началось во второй четверти дня?

In [20]:
df.filter((hour(df['trip_start_timestamp']) >= 6) & (hour(df['trip_start_timestamp']) < 12)).count()

538737

№12 Найдите топ три даты, в которые было суммарно больше всего чаевых? (Чаевые выдаются после совершения поездки)
<br> Ожидаемый формат дат YYYY-MM-DD
<br>Вам может понадобится конвертация типов <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast">cast</a>

In [22]:
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_date

In [30]:
df.groupBy(to_date(df['trip_start_timestamp'])).agg({"tips": "sum"}).orderBy('sum(tips)', ascending=False).show(5)

+-------------------------------+------------------+
|to_date(`trip_start_timestamp`)|         sum(tips)|
+-------------------------------+------------------+
|                     2016-11-03|109894.00000000015|
|                     2016-11-09| 106329.8699999999|
|                     2016-11-16|100253.60000000037|
|                     2016-12-01|  98208.2300000001|
|                     2016-11-10| 97594.29000000063|
+-------------------------------+------------------+
only showing top 5 rows



№13 Сколько было заказов в дату с наибольшим спросом?

In [32]:
df.groupBy(to_date(df['trip_start_timestamp'])).count().orderBy('count', ascending=False).show(5)

+-------------------------------+-----+
|to_date(`trip_start_timestamp`)|count|
+-------------------------------+-----+
|                     2016-11-03|61259|
|                     2016-12-16|59137|
|                     2016-12-09|58583|
|                     2016-12-15|57084|
|                     2016-11-04|56800|
+-------------------------------+-----+
only showing top 5 rows



Подгрузите данные о марках машин из датасета <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week3/spark-tasks/taxi_cars_data.parquet">taxi_cars_data.parquet</a>

In [33]:
df_car = spark.read.parquet('taxi_cars_data.parquet')

In [34]:
df_car.show()

+-------+-------------------+
|taxi_id|          car_model|
+-------+-------------------+
|   1159|       Toyota Prius|
|   7273|Ford Crown Victoria|
|   2904|        Honda Civic|
|   3210|        Ford Fusion|
|   2088|       Toyota Camry|
|   4821|Ford Crown Victoria|
|   2069|    Hyundai Elantra|
|   6240|       Toyota Camry|
|    296|     Hyundai Sonata|
|   2136|Ford Crown Victoria|
|   1436|     Toyota Corolla|
|   1090|     Toyota Corolla|
|   7711|        Lincoln MKZ|
|   1572|Ford Crown Victoria|
|   3959|     Chevrolet Aveo|
|   5645|  Chevrolet Lacetti|
|   5149|            Audi A7|
|   6366|     Hyundai Sonata|
|    451|     Hyundai Accent|
|   1669|           Kia Ceed|
+-------+-------------------+
only showing top 20 rows



№14 Какая марка машины самая распрастранненая среди таксистов?
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.split">split</a>

In [43]:
from pyspark.sql.functions import split

In [57]:
df_car.groupBy(split(df_car['car_model'], ' ')[0])\
.count().orderBy('count', ascending=False).show(5)

+--------------------------+-----+
|split(car_model,  , -2)[0]|count|
+--------------------------+-----+
|                      Ford| 1484|
|                   Hyundai|  792|
|                    Toyota|  691|
|                 Chevrolet|  473|
|                       Kia|  265|
+--------------------------+-----+
only showing top 5 rows



№15 Сколько раз и какая модель машин чаще всего встречается в поездках?
<br>Подробнее <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.join">join</a>

In [64]:
df.join(df_car, df.taxi_id == df_car.taxi_id).groupBy(df_car['car_model'])\
.count().orderBy('count', ascending=False).show(5)
    

+-------------------+------+
|          car_model| count|
+-------------------+------+
|Ford Crown Victoria|388682|
|     Hyundai Accent|150764|
|           Kia Ceed|143649|
|     Hyundai Sonata|141570|
|        Ford Mondeo|135466|
+-------------------+------+
only showing top 5 rows



Почувствуй силу сжатия! сохрани DataFrame в csv и сравни размеры файлов.

In [ ]:
# Ваш код с coalesce(1)

Теперь загрузите данные из csv и проверьте типы методом printSchema().

In [ ]:
# Ваш код с printSchema() для DataFrame из csv

Не забудьте посетить SparkUI и изучить историю ваших задач.

In [119]:
spark